In [ ]:
import os
import spikeinterface.extractors as se
import spikeinterface.full as si
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.postprocessing as spost
import spikeinterface.qualitymetrics as sqm
import spikeinterface.comparison as sc
import spikeinterface.exporters as sexp
import spikeinterface.widgets as sw
import probeinterface as pi
from probeinterface import Probe, ProbeGroup
from probeinterface.plotting import plot_probe, plot_probe_group
from probeinterface import generate_multi_columns_probe
import pyintan
import matplotlib.pyplot as plt
import numpy as np
import warnings
import re
from pathlib import Path
import random 
import pandas as pd
import mountainsort5 as ms5
all_rec_lists = []
all_names = []
all_chan = []
all_type = []

main_folder = Path("E:\Animal_recordings_all\depth_4ch\LKO_depth_4ch\LKO_Chronic\LKO_Chronic_2")
subfolders = [f.path for f in os.scandir(main_folder) if f.is_dir()]

hours_needed = 20
minutes_per_hour = 60
total_minutes_needed = hours_needed * minutes_per_hour

for subfolder in subfolders:
    print(f"\nContents of {subfolder}:")
    
    list_of_files = os.listdir(subfolder)
    list_of_files = [file for file in list_of_files if 'merged' not in file]
    list_of_files = [file for file in list_of_files if 'settings' not in file]
    error_files = []
    print(len(list_of_files))
    
    total_minutes_collected = 0
    subfolder_rec_list = []

    while total_minutes_collected < total_minutes_needed and len(list_of_files) > 0:
        remaining_minutes_needed = total_minutes_needed - total_minutes_collected
        list_of_files_subset = list_of_files[:min(remaining_minutes_needed, 60)]
        
        print(f"Processing these files: {list_of_files_subset}")
        
        list_of_recordings = []
        for file_name in list_of_files_subset:
            file_path = os.path.join(subfolder, file_name)
            try:
                recording = se.read_intan(file_path, stream_name='RHD2000 amplifier channel', use_names_as_ids=True)  # Assuming data is float32
                list_of_recordings.append(recording)
            except Exception as e:
                print(f"Error reading file {file_name}: {e}")
                error_files.append(file_name)
                continue

        if len(list_of_recordings) > 0:
            try:
                concatenated_recording = si.concatenate_recordings(list_of_recordings)
                rec_cmr = spre.common_reference(concatenated_recording, operator="median", reference="global")  # Rereferencing the data
                recording_notch_ecog = spre.notch_filter(rec_cmr, q=50)  # Notch filter
                recording_bp = spre.bandpass_filter(recording_notch_ecog, freq_min=300, freq_max=6000)  # Bandpass filter
                recording_resampled_ecog = recording_bp

                subfolder_rec_list.append(recording_resampled_ecog)
                
                channel_ids = recording_resampled_ecog.get_channel_ids()
                fs = recording_resampled_ecog.get_sampling_frequency()
                num_chan = recording_resampled_ecog.get_num_channels()
                num_segments = recording_resampled_ecog.get_num_segments()
                
                print("Channel_ids =", channel_ids)
                print(f"Sampling_frequency = {fs}")
                print("Number of Channels =", num_chan)
                print("Number of segments =", num_segments)
                print('Total_rec_duration =', recording_resampled_ecog.get_total_duration())

                k = subfolder.split("/")[-1]
                name = k.split("\\")[-1]

                
                all_chan.append(num_chan)

                if re.search(r'lko|LKO|L', name):
                    type = 'LKO'
                    print(f"Type: {type}")
                else:
                    type = 'WT'
                    print("Type: Not matched")
                
                all_type.append(type)
                
                total_minutes_collected += len(list_of_recordings)

            except ValueError as ve:
                print(f"Concatenation error in folder {subfolder}: {ve}")
                continue

        list_of_files = list_of_files[len(list_of_files_subset):]
    all_names.append(name)
    if total_minutes_collected >= total_minutes_needed:
        print(f"Successfully created {hours_needed} hours of recordings.")
    else:
        print(f"Collected {total_minutes_collected} minutes of recordings.")
    
    # Add the subfolder's recordings list to the main list
    all_rec_lists.append(subfolder_rec_list)

print(f"Error files: {error_files}")

In [ ]:
hour = ['first_60', 'second_60', 'third_60', 'fourth_60', 'fifth_60', 'sixth_60', 'seventh_60', 'eighth_60', 'ninth_60', 'tenth_60', 'eleventh_60', 'twelfth_60', 'thirteenth_60', 'fourteenth_60', 'fifteenth_60', 'sixteenth_60', 'seventeenth_60', 'eighteenth_60', 'nineteenth_60', 'twentieth_60']

df_all = pd.DataFrame()
g = 1
all_sorting=[]
for i in range(len(all_rec_lists)):
     for j in range(len(all_rec_lists[i])):
        print(i,j)
        ch = all_rec_lists[i][j].get_num_channels()
        print(i, j)
        if ch >= 4:
            sliced_ch = all_rec_lists[i][j].get_channel_ids()
            sliced_ch = sliced_ch[:4]
            #sliced_ch = sliced_ch[4:9]
            all_rec_lists[i][j] = all_rec_lists[i][j].channel_slice(sliced_ch)
            
            #if ch == 4:
            ch = all_rec_lists[i][j].get_num_channels()
            dur = all_rec_lists[i][j].get_total_duration()
            
           
            kk = all_rec_lists[i][j]
            #kk = se.read_intan(kk, stream_name='RHD2000 amplifier channel', use_names_as_ids=True)
            recording_resampled_ecog = kk
           
            probe = generate_multi_columns_probe(num_columns=3,
                             num_contact_per_column=[1, 2, 1],
                             xpitch=75, ypitch=155, y_shift_per_column=[0, -75, 0],
                             contact_shapes='square', contact_shape_params={'width' : 20 , 'height' :20 })

            channel_indices = np.array([0, 1, 3, 2])
            probe.set_device_channel_indices(channel_indices)
            print(probe.device_channel_indices)
            
            contact_ids = np.array([0, 1, 3, 2]) 
            probe.set_contact_ids(contact_ids)
            
            rec_2.set_probe(probe, in_place = True)
            rec_2.get_channel_locations()
            plot_probe(probe, with_contact_id=True, with_device_index=True)
                    
            
            rec_normed = spre.zscore(recording=recording_resampled_ecog)
            rec_2 = spre.whiten(rec_normed) 
            
                  
            sorting_rec = ms5.sorting_scheme2(rec_2,
                sorting_parameters=ms5.Scheme2SortingParameters(
                    phase1_detect_channel_radius=150,
                    detect_channel_radius=50))        
            
            all_sorting.append(sorting_rec)
            
            print("Sorter found", len(sorting_rec.get_unit_ids()), "units")
            sorting_rec = sorting_rec.remove_empty_units()
            print("Sorter found", len(sorting_rec.get_unit_ids()), "non empty units")
            
            recording_resampled_ecog.annotate(is_filtered=True) # since we filtered the rec 
    
            if len(sorting_rec.get_unit_ids()) >0 :
                print("non-zero-clusters")
                
                print(f"Processing: {name}, Type: {type}, Recording Index: {j}")

                waveform_folder = Path(r"/home/ayandas22/Animal_recordings_all/LKO_depth_4ch/LKO_Chronic/LKO_Chronic_2")
                waveform_folder = waveform_folder / f"waveforms_kshtj_{all_names[i]}_{hour[j]}" 
            
                waveform_folder.mkdir(parents=True, exist_ok=True)

                type = all_type[i]  
                print(f"Processing {name} at index {i}")

                we = si.extract_waveforms(rec_2, sorting_rec, folder=waveform_folder, load_if_exists=False, overwrite=True,sparse=False) 
                spost.compute_spike_amplitudes(we)
            
                unit_ids = sorting_rec.unit_ids
                plt.figure(figsize=(12, 8))
                sw.plot_unit_templates(we, unit_ids=unit_ids)
                plt.suptitle(f"{all_names[i]}_{hour[j]}", fontsize=12, y=0.01)
                plt.tight_layout(rect=[0, 0.05, 1, 1])
                plots_directory =  Path(r"/home/ayandas22/Animal_recordings_all/LKO_depth_4ch/LKO_Chronic/LKO_Chronic_2")
                plots_directory.mkdir(parents=True, exist_ok=True) 
                plt.savefig(plots_directory / f"{all_names[i]}_{hour[j]}_xx.png", bbox_inches='tight')
               




In [ ]:
import os
import spikeinterface.extractors as se
import spikeinterface.full as si
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.postprocessing as spost
import spikeinterface.qualitymetrics as sqm
import spikeinterface.comparison as sc
import spikeinterface.exporters as sexp
import spikeinterface.widgets as sw
import probeinterface as pi
from probeinterface import Probe, ProbeGroup
from probeinterface.plotting import plot_probe, plot_probe_group
from probeinterface import generate_multi_columns_probe
import pyintan
import matplotlib.pyplot as plt
import numpy as np
import warnings
import re
from pathlib import Path
import random 
import pandas as pd
import mountainsort5 as ms5
all_rec_lists = []
all_names = []
all_chan = []
all_type = []

main_folder = Path(r"E:\Animal_recordings_all\depth_4ch\LKO_depth_4ch\LKO_Chronic\LKO_chronic_3")
subfolders = [f.path for f in os.scandir(main_folder) if f.is_dir()]

hours_needed = 20
minutes_per_hour = 60
total_minutes_needed = hours_needed * minutes_per_hour

for subfolder in subfolders:
    #print(f"\nContents of {subfolder}:")
    
    list_of_files = os.listdir(subfolder)
    list_of_files = [file for file in list_of_files if 'merged' not in file]
    list_of_files = [file for file in list_of_files if 'settings' not in file]
    error_files = []
    print(len(list_of_files))
    
    total_minutes_collected = 0
    subfolder_rec_list = []

    while total_minutes_collected < total_minutes_needed and len(list_of_files) > 0:
        remaining_minutes_needed = total_minutes_needed - total_minutes_collected
        list_of_files_subset = list_of_files[:min(remaining_minutes_needed, 60)]
        
        #print(f"Processing these files: {list_of_files_subset}")
        
        list_of_recordings = []
        for file_name in list_of_files_subset:
            file_path = os.path.join(subfolder, file_name)
            try:
                recording = se.read_intan(file_path, stream_name='RHD2000 amplifier channel', use_names_as_ids=True)  # Assuming data is float32
                list_of_recordings.append(recording)
            except Exception as e:
                print(f"Error reading file {file_name}: {e}")
                error_files.append(file_name)
                continue

        if len(list_of_recordings) > 0:
            try:
                concatenated_recording = si.concatenate_recordings(list_of_recordings)
                rec_cmr = spre.common_reference(concatenated_recording, operator="median", reference="global")  # Rereferencing the data
                recording_notch_ecog = spre.notch_filter(rec_cmr, q=50)  # Notch filter
                recording_bp = spre.bandpass_filter(recording_notch_ecog, freq_min=300, freq_max=6000)  # Bandpass filter
                recording_resampled_ecog = recording_bp

                subfolder_rec_list.append(recording_resampled_ecog)
                
                channel_ids = recording_resampled_ecog.get_channel_ids()
                fs = recording_resampled_ecog.get_sampling_frequency()
                num_chan = recording_resampled_ecog.get_num_channels()
                num_segments = recording_resampled_ecog.get_num_segments()
                
                #print("Channel_ids =", channel_ids)
                #print(f"Sampling_frequency = {fs}")
                #print("Number of Channels =", num_chan)
                #print("Number of segments =", num_segments)
               # print('Total_rec_duration =', recording_resampled_ecog.get_total_duration())

                k = subfolder.split("/")[-1]
                name = k.split("\\")[-1]

                
                all_chan.append(num_chan)

                if re.search(r'lko|LKO|L', name):
                    type = 'LKO'
                    print(f"Type: {type}")
                else:
                    type = 'WT'
                    print("Type: Not matched")
                
                all_type.append(type)
                
                total_minutes_collected += len(list_of_recordings)

            except ValueError as ve:
                print(f"Concatenation error in folder {subfolder}: {ve}")
                continue

        list_of_files = list_of_files[len(list_of_files_subset):]
    all_names.append(name)
    if total_minutes_collected >= total_minutes_needed:
        G=1
        #print(f"Successfully created {hours_needed} hours of recordings.")
    else:
        F=1
        #print(f"Collected {total_minutes_collected} minutes of recordings.")
    
    
    all_rec_lists.append(subfolder_rec_list)

print(f"Error files: {error_files}")


hour = ['first_60', 'second_60', 'third_60', 'fourth_60', 'fifth_60', 'sixth_60', 'seventh_60', 'eighth_60', 'ninth_60', 'tenth_60', 'eleventh_60', 'twelfth_60', 'thirteenth_60', 'fourteenth_60', 'fifteenth_60', 'sixteenth_60', 'seventeenth_60', 'eighteenth_60', 'nineteenth_60', 'twentieth_60']

df_all = pd.DataFrame()
g = 1
all_sorting=[]
for i in range(len(all_rec_lists)):
     for j in range(len(all_rec_lists[i])):
        print(i,j)
        ch = all_rec_lists[i][j].get_num_channels()
        print(i, j)
        if ch >= 4:
            sliced_ch = all_rec_lists[i][j].get_channel_ids()
            sliced_ch = sliced_ch[:4]
            #sliced_ch = sliced_ch[4:9]
            all_rec_lists[i][j] = all_rec_lists[i][j].channel_slice(sliced_ch)
            
            #if ch == 4:
            ch = all_rec_lists[i][j].get_num_channels()
            dur = all_rec_lists[i][j].get_total_duration()
            
           
            kk = all_rec_lists[i][j]
            #kk = se.read_intan(kk, stream_name='RHD2000 amplifier channel', use_names_as_ids=True)
            recording_resampled_ecog = kk
           
            probe = generate_multi_columns_probe(num_columns=3,
                             num_contact_per_column=[1, 2, 1],
                             xpitch=75, ypitch=155, y_shift_per_column=[0, -75, 0],
                             contact_shapes='square', contact_shape_params={'width' : 20 , 'height' :20 })

            channel_indices = np.array([0, 1, 3, 2])
            probe.set_device_channel_indices(channel_indices)
            print(probe.device_channel_indices)
            
            contact_ids = np.array([0, 1, 3, 2]) 
            probe.set_contact_ids(contact_ids)
            rec_normed = spre.zscore(recording=recording_resampled_ecog)
            rec_2 = spre.whiten(rec_normed) 


            rec_2.set_probe(probe, in_place = True)
            rec_2.get_channel_locations()
            #plot_probe(probe, with_contact_id=True, with_device_index=True)
                    
                        
                  
            sorting_rec = ms5.sorting_scheme2(rec_2,
                sorting_parameters=ms5.Scheme2SortingParameters(
                    phase1_detect_channel_radius=150,
                    detect_channel_radius=50))        
            
            all_sorting.append(sorting_rec)
            
            #print("Sorter found", len(sorting_rec.get_unit_ids()), "units")
            sorting_rec = sorting_rec.remove_empty_units()
            #print("Sorter found", len(sorting_rec.get_unit_ids()), "non empty units")
            
            recording_resampled_ecog.annotate(is_filtered=True) # since we filtered the rec 
    
            if len(sorting_rec.get_unit_ids()) >0 :
                print("non-zero-clusters")
                
                print(f"Processing: {name}, Type: {type}, Recording Index: {j}")

                waveform_folder = Path(r"E:\Animal_recordings_all\depth_4ch\LKO_depth_4ch\LKO_Chronic\LKO_chronic_1")
                waveform_folder = waveform_folder / f"waveforms_kshtj_{all_names[i]}_{hour[j]}" 
            
                waveform_folder.mkdir(parents=True, exist_ok=True)

                type = all_type[i]  
                print(f"Processing {name} at index {i}")

                we = si.extract_waveforms(rec_2, sorting_rec, folder=waveform_folder, load_if_exists=False, overwrite=True,sparse=False) 
                spost.compute_spike_amplitudes(we)
            
                unit_ids = sorting_rec.unit_ids
                plt.figure(figsize=(12, 8))
                #sw.plot_unit_templates(we, unit_ids=unit_ids)
                plt.suptitle(f"{all_names[i]}_{hour[j]}", fontsize=12, y=0.01)
                plt.tight_layout(rect=[0, 0.05, 1, 1])
                plots_directory =  Path(r"E:\Animal_recordings_all\depth_4ch\LKO_depth_4ch\LKO_Chronic\LKO_chronic_1")
                plots_directory.mkdir(parents=True, exist_ok=True) 
                plt.savefig(plots_directory / f"{all_names[i]}_{hour[j]}_xx.png", bbox_inches='tight')
               



1
Type: LKO
205
Type: LKO
Type: LKO
Type: LKO
Type: LKO
36
Type: LKO
1
Type: LKO
8
Type: LKO
35
Error reading file chronic_2nd_lko_day_18march_240318_174849.rhs: Size of available data is not a multiple of the data-type size.
Type: LKO
835
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
74
Type: LKO
Type: LKO
7
Type: LKO
1
Type: LKO
262
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
13
Type: LKO
1
Type: LKO
536
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Type: LKO
Error reading file chronic_2nd_lko_day5_13march_2024_240313_210831.rhs: 'utf-16-le' codec can't decode bytes in position 372-373: illegal encoding
Error reading file chronic_2nd_lko_day5_13march_2024_240313_210931.rhs: 'utf-16-le' codec can't decode bytes in position 5738-5739: illegal encoding
Error reading file chronic_2nd_lko_day5_13march_2024_240313_211031.rhs: 'utf-16-le' codec can't decode bytes in po

C:\Users\garim\AppData\Local\Temp\ipykernel_68576\2406407112.py:195: DeprecationWarning: load_if_exists=True/false is deprcated. Use load_waveforms() instead.
  we = si.extract_waveforms(rec_2, sorting_rec, folder=waveform_folder, load_if_exists=False, overwrite=True,sparse=False)


extract waveforms memmap multi buffer:   0%|          | 0/37 [00:00<?, ?it/s]

extract amplitudes:   0%|          | 0/37 [00:00<?, ?it/s]

1 0
1 0
[0 1 3 2]
Using the full recording for training: 3600.128 sec
Running phase 1 sorting
Number of channels: 4
Number of timepoints: 108003840
Sampling frequency: 30000.0 Hz
Channel 0: [0. 0.]
Channel 1: [ 75. -75.]
Channel 2: [150.   0.]
Channel 3: [75. 80.]
Loading traces
*** MS5 Elapsed time for load_traces: 110.118 seconds ***
Detecting spikes

Adjacency for detect spikes with channel radius 150
[[0, 1, 2, 3], [0, 1, 2], [0, 1, 2, 3], [0, 2, 3]]

m = 0 (nbhd size: 4)
m = 1 (nbhd size: 3)
m = 2 (nbhd size: 4)
m = 3 (nbhd size: 3)
Detected 84070 spikes
*** MS5 Elapsed time for detect_spikes: 3.080 seconds ***
Removing duplicate times
*** MS5 Elapsed time for remove_duplicate_times: 0.009 seconds ***
Extracting 81226 snippets
*** MS5 Elapsed time for extract_snippets: 0.081 seconds ***
Computing PCA features with npca=12
*** MS5 Elapsed time for compute_pca_features: 0.060 seconds ***
Isosplit6 clustering with npca_per_subdivision=10
Found 3 clusters
*** MS5 Elapsed time for isos

...

6 6
6 6
[0 1 3 2]
Using the full recording for training: 3600.128 sec
Running phase 1 sorting
Number of channels: 4
Number of timepoints: 108003840
Sampling frequency: 30000.0 Hz
Channel 0: [0. 0.]
Channel 1: [ 75. -75.]
Channel 2: [150.   0.]
Channel 3: [75. 80.]
Loading traces
*** MS5 Elapsed time for load_traces: 105.986 seconds ***
Detecting spikes

Adjacency for detect spikes with channel radius 150
[[0, 1, 2, 3], [0, 1, 2], [0, 1, 2, 3], [0, 2, 3]]

m = 0 (nbhd size: 4)
m = 1 (nbhd size: 3)
m = 2 (nbhd size: 4)
m = 3 (nbhd size: 3)
Detected 239496 spikes
*** MS5 Elapsed time for detect_spikes: 4.195 seconds ***
Removing duplicate times
*** MS5 Elapsed time for remove_duplicate_times: 0.002 seconds ***
Extracting 239496 snippets
*** MS5 Elapsed time for extract_snippets: 0.227 seconds ***
Computing PCA features with npca=12
*** MS5 Elapsed time for compute_pca_features: 0.060 seconds ***
Isosplit6 clustering with npca_per_subdivision=10
Found 234 clusters
*** MS5 Elapsed time for 

C:\Users\garim\AppData\Local\Temp\ipykernel_68576\2406407112.py:195: DeprecationWarning: load_if_exists=True/false is deprcated. Use load_waveforms() instead.
  we = si.extract_waveforms(rec_2, sorting_rec, folder=waveform_folder, load_if_exists=False, overwrite=True,sparse=False)


extract waveforms memmap multi buffer:   0%|          | 0/3601 [00:00<?, ?it/s]

extract amplitudes:   0%|          | 0/3601 [00:00<?, ?it/s]

6 7
6 7
[0 1 3 2]
Using the full recording for training: 3600.128 sec
Running phase 1 sorting
Number of channels: 4
Number of timepoints: 108003840
Sampling frequency: 30000.0 Hz
Channel 0: [0. 0.]
Channel 1: [ 75. -75.]
Channel 2: [150.   0.]
Channel 3: [75. 80.]
Loading traces
*** MS5 Elapsed time for load_traces: 107.658 seconds ***
Detecting spikes

Adjacency for detect spikes with channel radius 150
[[0, 1, 2, 3], [0, 1, 2], [0, 1, 2, 3], [0, 2, 3]]

m = 0 (nbhd size: 4)
m = 1 (nbhd size: 3)
m = 2 (nbhd size: 4)
m = 3 (nbhd size: 3)
Detected 260462 spikes
*** MS5 Elapsed time for detect_spikes: 5.086 seconds ***
Removing duplicate times
*** MS5 Elapsed time for remove_duplicate_times: 0.006 seconds ***
Extracting 260460 snippets
*** MS5 Elapsed time for extract_snippets: 0.333 seconds ***
Computing PCA features with npca=12
*** MS5 Elapsed time for compute_pca_features: 0.089 seconds ***
Isosplit6 clustering with npca_per_subdivision=10
